In [1]:
from citipy import citipy
import pandas as pd
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
from config import weather_api
from pprint import pprint

# Creating the cities DataFrame

In [2]:
#Create a list of random latitudes and longitudes
cities_df = pd.DataFrame({"lat":np.random.uniform(-90,90,500), "long":np.random.uniform(-180,180,500)})

print(len(cities_df))

cities_df.head()

500


,lat,long
0,48.354516,132.377279
1,-87.668421,133.629866
2,58.790490,170.444256
3,36.326503,67.393280
4,-34.175230,178.162245


In [3]:
city_name = []
country_list =[]

for coords in range(0, len(cities_df["lat"])):
    lat=cities_df.iloc[coords]["lat"]
    long=cities_df.iloc[coords]["long"]
    city = citipy.nearest_city(lat,long)
    name= city.city_name
    city_name.append(name)
    country= city.country_code
    country_list.append(country)

cities_df["city"]=city_name
cities_df["country"]=country_list

cities_df.head()

,lat,long,city,country
0,48.354516,132.377279,babstovo,ru
1,-87.668421,133.629866,hobart,au
2,58.790490,170.444256,tilichiki,ru
3,36.326503,67.393280,kholm,af
4,-34.175230,178.162245,whitianga,nz


In [4]:
cities_df["city"].duplicated().value_counts()

False    296
True     204
Name: city, dtype: int64

### Since there are duplicated cities in our DF, we  have to drop those specific rows. 

In [5]:
cleancities_df = cities_df.drop_duplicates(subset="city", keep="first").reset_index(drop=True)


In [6]:
len(cleancities_df)

296

### Now that we have a clean data set, we can keep going with the exercise.

In [7]:
cleancities_df["max temp"] = ""
cleancities_df["humidity"] = ""
cleancities_df["cloudiness"] = ""
cleancities_df["wind speed"] = ""
cleancities_df["date"] = ""

cleancities_df.head()

,lat,long,city,country,max temp,humidity,cloudiness,wind speed,date
0,48.354516,132.377279,babstovo,ru,,,,,
1,-87.668421,133.629866,hobart,au,,,,,
2,58.790490,170.444256,tilichiki,ru,,,,,
3,36.326503,67.393280,kholm,af,,,,,
4,-34.175230,178.162245,whitianga,nz,,,,,


In [8]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
api_key = weather_api
query_url = f"{url}appid={api_key}&units={units}&q="

for index, row in cleancities_df.iterrows():
    city = row["city"]
    response = requests.get(query_url + city).json()
    
    print(f"Looking for the weather in city number {index}, {city}")
          
    try:
          cleancities_df.at[index, "max temp"] = response["main"]["temp_max"]
          cleancities_df.at[index, "humidity"] = response["main"]["humidity"]
          cleancities_df.at[index, "cloudiness"] = response["clouds"]["all"]
          cleancities_df.at[index, "wind speed"] = response["wind"]["speed"]
          cleancities_df.at[index, "date"] = response["dt"]
    except:
          print("Missing city. Moving on...")
          

cleancities_df.head()

Looking for the weather in city number 0, babstovo
Looking for the weather in city number 1, hobart
Looking for the weather in city number 2, tilichiki
Looking for the weather in city number 3, kholm
Looking for the weather in city number 4, whitianga
Looking for the weather in city number 5, deputatskiy
Looking for the weather in city number 6, new norfolk
Looking for the weather in city number 7, klaksvik
Looking for the weather in city number 8, kodiak
Looking for the weather in city number 9, tautira
Looking for the weather in city number 10, brae
Looking for the weather in city number 11, saleaula
Missing city. Moving on...
Looking for the weather in city number 12, caravelas
Looking for the weather in city number 13, rikitea
Looking for the weather in city number 14, harper
Looking for the weather in city number 15, busselton
Looking for the weather in city number 16, jamestown
Looking for the weather in city number 17, kapaa
Looking for the weather in city number 18, teguldet
Lo

Looking for the weather in city number 150, faanui
Looking for the weather in city number 151, port hedland
Looking for the weather in city number 152, peruibe
Looking for the weather in city number 153, sibu
Looking for the weather in city number 154, butaritari
Looking for the weather in city number 155, banda aceh
Looking for the weather in city number 156, agboville
Looking for the weather in city number 157, hambantota
Looking for the weather in city number 158, biak
Looking for the weather in city number 159, minab
Looking for the weather in city number 160, talaya
Looking for the weather in city number 161, sentyabrskiy
Missing city. Moving on...
Looking for the weather in city number 162, saint-philippe
Looking for the weather in city number 163, jasper
Looking for the weather in city number 164, flin flon
Looking for the weather in city number 165, torbay
Looking for the weather in city number 166, nurota
Looking for the weather in city number 167, bulaevo
Looking for the weat

,lat,long,city,country,max temp,humidity,cloudiness,wind speed,date
0,48.354516,132.377279,babstovo,ru,70.47,72,0,11.61,1599272013
1,-87.668421,133.629866,hobart,au,59,33,20,14.99,1599271264
2,58.790490,170.444256,tilichiki,ru,49.57,53,0,13.11,1599272013
3,36.326503,67.393280,kholm,af,51.55,98,77,5.17,1599272013
4,-34.175230,178.162245,whitianga,nz,62.01,66,59,7,1599272014


In [13]:
print(len(cleancities_df["city"]))

296


In [28]:
cleancities_df.isnull().sum()

lat            0
long           0
city           0
country        0
max temp      26
humidity      26
cloudiness    26
wind speed    26
date           0
dtype: int64

In [31]:
cleancities_df.dropna(inplace=True)

cleancities_df.isnull().sum()


lat           0
long          0
city          0
country       0
max temp      0
humidity      0
cloudiness    0
wind speed    0
date          0
dtype: int64

In [32]:
print(len(cleancities_df["city"]))

270


In [9]:
cleancities_df.dtypes

lat           float64
long          float64
city           object
country        object
max temp       object
humidity       object
cloudiness     object
wind speed     object
date           object
dtype: object

In [12]:
cleancities_df["max temp"] = pd.to_numeric(cleancities_df["max temp"])
cleancities_df["humidity"] = pd.to_numeric(cleancities_df["humidity"])
cleancities_df["cloudiness"] = pd.to_numeric(cleancities_df["cloudiness"])
cleancities_df["wind speed"] = pd.to_numeric(cleancities_df["wind speed"])

cleancities_df.dtypes

lat           float64
long          float64
city           object
country        object
max temp      float64
humidity      float64
cloudiness    float64
wind speed    float64
date           object
dtype: object